<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-행정동-/-법정동-가져올-좌표-리스트-생성" data-toc-modified-id="1.-행정동-/-법정동-가져올-좌표-리스트-생성-1">1. 행정동 / 법정동 가져올 좌표 리스트 생성</a></span></li><li><span><a href="#2.-카카오-API-활용---행정동-/-법정동-검색" data-toc-modified-id="2.-카카오-API-활용---행정동-/-법정동-검색-2">2. 카카오 API 활용 - 행정동 / 법정동 검색</a></span></li></ul></div>

In [ ]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas
import re

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
지하철 = pd.read_csv('입지선정후보군_좌표추가.csv', encoding='cp949')
print(지하철.shape)
지하철.head()

## 1. 행정동 / 법정동 가져올 좌표 리스트 생성

In [8]:
XY = np.array([list(i) for i in zip(버스['X좌표'], 버스['Y좌표'])])
XY

array([[126.97277266,  37.55547526],
       [127.05286142,  37.48227641],
       [126.94470755,  37.51388591],
       ...,
       [127.07652825,  37.58842234],
       [127.08499787,  37.58768876],
       [127.09420247,  37.58783786]])

## 2. 카카오 API 활용 - 행정동 / 법정동 검색

In [17]:
# 행정동 / 법정동 검색 함수 정의
def region_name(xy_list):
    url = f"https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={xy_list[0]}&y={xy_list[1]}"
    result = requests.get(urlparse(url).geturl(),
                          headers={"Authorization":"KakaoAK 9366298e2a42e074b458eb183e2a78ef"})
    json_obj = result.json()
    global val   # 전역 변수를 지역 범위에서 사용하고 싶을 경우
    for document in json_obj['documents']:
        # 값이 NoneType 일 경우의 오류 방지
        try:
            val = [document['region_type'], document['region_3depth_name'], document['x'], document['y']]
        except:
            val = [0,0,0,0]
    return val

In [19]:
region_name([126.972773, 37.555475])

['H', '회현동', 126.97935667015034, 37.55729358571407]

In [ ]:
# 행정동 / 법정동 검색 함수 반복 적용
list1 = []
for i in XY:
    list1.append(region_name(i))

In [ ]:
버스2 = pd.DataFrame(list1, columns=['H/B', '행정동', '행정동_경도', '행정동_위도'])
버스2.head()

In [ ]:
버스3 = pd.concat([버스, 버스2], axis=1)
print(버스3.shape)
버스3.head()